# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [118]:
import numpy as np
import pandas as pd
import time
import xlsxwriter
import requests
from scipy import stats
import math
from statistics import mean

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secretsRENAMED import FINNHUB_API_KEY

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [12]:
symbol = 'AAPL'

pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={symbol}&token={FINNHUB_API_KEY}'
basic_financials_api_url = f'https://finnhub.io/api/v1/stock/metric?symbol={symbol}&metric=all&token={FINNHUB_API_KEY}'

pricing_data = requests.get(pricing_api_url).json()
basic_financials_data = requests.get(basic_financials_api_url).json()

print(pricing_data)
print(basic_financials_data)

{'c': 273.76, 'd': 0.36, 'dp': 0.1317, 'h': 274.36, 'l': 272.35, 'o': 272.69, 'pc': 273.4, 't': 1767042000}
{'metric': {'10DayAverageTradingVolume': 47.96383, '13WeekPriceReturnDaily': 6.4352, '26WeekPriceReturnDaily': 36.4953, '3MonthADReturnStd': 17.600784, '3MonthAverageTradingVolume': 48.90346, '52WeekHigh': 288.62, '52WeekHighDate': '2025-12-03', '52WeekLow': 169.2101, '52WeekLowDate': '2025-04-08', '52WeekPriceReturnDaily': 5.8869, '5DayPriceReturnDaily': -0.0987, 'assetTurnoverAnnual': 1.1584, 'assetTurnoverTTM': 1.2186, 'beta': 1.0966666, 'bookValuePerShareAnnual': 4.991, 'bookValuePerShareQuarterly': 4.991, 'bookValueShareGrowth5Y': 5.34, 'capexCagr5Y': 11.71, 'cashFlowPerShareAnnual': 6.6855, 'cashFlowPerShareQuarterly': 6.6855, 'cashFlowPerShareTTM': 6.86253, 'cashPerSharePerShareAnnual': 3.7024, 'cashPerSharePerShareQuarterly': 3.7024, 'currentDividendYieldTTM': 0.3822, 'currentEv/freeCashFlowAnnual': 41.4856, 'currentEv/freeCashFlowTTM': 41.4856, 'currentRatioAnnual': 0.89

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [ ]:
price = pricing_data['c']
pe_ratio = basic_financials_data['metric']['peAnnual']

36.0207

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [17]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [34]:
final_dataframe = pd.DataFrame(columns = my_columns)
rows = []
for stock in stocks['Ticker']:
    try:
        pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}'
        basic_financials_api_url = f'https://finnhub.io/api/v1/stock/metric?symbol={stock}&metric=all&token={FINNHUB_API_KEY}'

        pricing_data = requests.get(pricing_api_url).json()
        if pricing_data.get('error'):
            raise Exception(f"Error fetching pricing data for {stock}: {pricing_data['error']}")

        basic_financials = requests.get(basic_financials_api_url).json()
        if basic_financials.get('error'):
            raise Exception(f"Error fetching basic financials data for {stock}: {basic_financials['error']}")

        price = pricing_data.get('c') or 'N/A'
        pe_annual = basic_financials.get('metric', {}).get('peAnnual') or 'N/A'

        price = price if price != 0 else 'N/A'
        pe_annual = pe_annual if pe_annual != 0 else 'N/A'

    except Exception as e:
        print(f"An error occurred for {stock}: {e}")
        price = 'N/A'
        pe_annual = 'N/A'
    
    rows.append([stock, price, pe_annual, 'N/A'])
    
    time.sleep(1)

final_dataframe = pd.DataFrame(rows, columns=my_columns)

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,137.22,29.6053,N/A
1,AAL,15.6509,11.9611,N/A
2,AAP,38.36,N/A,N/A
3,AAPL,270.15,35.8638,N/A
4,ABBV,229.74,94.3968,N/A
...,...,...,...,...
500,YUM,150.41,28.266,N/A
501,ZBH,90.37,19.7195,N/A
502,ZBRA,249.58,23.298,N/A
503,ZION,58.96,11.0241,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [44]:
final_dataframe_updated = final_dataframe[(final_dataframe['Price-to-Earnings Ratio'] != 'N/A') & (final_dataframe['Price'] != 'N/A')]

final_dataframe_updated = final_dataframe_updated[final_dataframe_updated['Price-to-Earnings Ratio'] > 0]

final_dataframe_updated = final_dataframe_updated.sort_values('Price-to-Earnings Ratio', ascending = True)
final_dataframe_updated = final_dataframe_updated[:50]
final_dataframe_updated.reset_index(inplace=True, drop=True)
final_dataframe_updated

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,LNC,44.71,2.5826,N/A
1,WU,9.33,3.1676,N/A
2,FMC,14.1955,5.0796,N/A
3,PVH,67.61,5.129,N/A
4,CHTR,215.169,5.9513,N/A
5,AES,14.6769,6.0821,N/A
6,CNC,41.83,6.1198,N/A
7,LEN,104.13,6.5268,N/A
8,DXC,14.36,6.558,N/A
9,CMCSA,29.905,6.7265,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [36]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [37]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [45]:
position_size = float(portfolio_size) / len(final_dataframe_updated.index)

for row in final_dataframe_updated.index:
    final_dataframe_updated.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe_updated.loc[row, 'Price'])

final_dataframe_updated

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,LNC,44.71,2.5826,447
1,WU,9.33,3.1676,2143
2,FMC,14.1955,5.0796,1408
3,PVH,67.61,5.129,295
4,CHTR,215.169,5.9513,92
5,AES,14.6769,6.0821,1362
6,CNC,41.83,6.1198,478
7,LEN,104.13,6.5268,192
8,DXC,14.36,6.558,1392
9,CMCSA,29.905,6.7265,668


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [120]:
stock = 'AAPL'

pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}'
basic_financials_api_url = f'https://finnhub.io/api/v1/stock/metric?symbol={stock}&metric=all&token={FINNHUB_API_KEY}'
financials_reported_api_url = f'https://finnhub.io/api/v1/stock/financials-reported?symbol={stock}&metric=all&token={FINNHUB_API_KEY}'

pricing_data = requests.get(pricing_api_url).json()
if pricing_data.get('error'):
    raise Exception(f"Error fetching pricing data for {stock}: {pricing_data['error']}")

basic_financials = requests.get(basic_financials_api_url).json()
if basic_financials.get('error'):
    raise Exception(f"Error fetching basic financials data for {stock}: {basic_financials['error']}")

financials_reported = requests.get(financials_reported_api_url).json()
if financials_reported.get('error'):
    raise Exception(f"Error fetching financials reported data for {stock}: {financials_reported['error']}")

price = pricing_data.get('c') or 'N/A'
pe_annual = basic_financials.get('metric', {}).get('peAnnual') or 'N/A'
pb = basic_financials.get('metric', {}).get('pb') or 'N/A'
ps_annual = basic_financials.get('metric', {}).get('psAnnual') or 'N/A'
ev_ebitda_ttm = basic_financials.get('metric', {}).get('evEbitdaTTM') or 'N/A'

ev = basic_financials.get('metric', {}).get('enterpriseValue') * 1000000 or 'N/A'
income_statement = financials_reported.get('data', [])[0].get('report', {}).get('ic', []) or 'N/A'

gross_profit_value = (
    next((item.get('value') for item in income_statement
          if item.get('concept') == 'us-gaap_GrossProfit'), 'N/A')
    if isinstance(income_statement, list) else 'N/A'
)

ev_gross_profit = ev / gross_profit_value if ev != 'N/A' and gross_profit_value != 'N/A' and gross_profit_value != 0 else 'N/A'

print(f"Price: {price}")
print(f"PE Annual: {pe_annual}")
print(f"PB: {pb}")
print(f"PS Annual: {ps_annual}")
print(f"EV/EBITDA TTM: {ev_ebitda_ttm}")

print(f"Enterprise Value: {ev}")
print(f"Gross Profit: {gross_profit_value}")
print(f"EV/Gross Profit: {ev_gross_profit}")

Price: 271.01
PE Annual: 35.7516
PB: 54.3114
PS Annual: 9.6226
EV/EBITDA TTM: 28.1612
Enterprise Value: 4067262000000
Gross Profit: 195201000000
EV/Gross Profit: 20.836276453501775


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [97]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/Gross Profit',
    'EV/Gross Profit Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

rows = []
for stock in stocks['Ticker']:
    try:
        pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}'
        basic_financials_api_url = f'https://finnhub.io/api/v1/stock/metric?symbol={stock}&metric=all&token={FINNHUB_API_KEY}'
        financials_reported_api_url = f'https://finnhub.io/api/v1/stock/financials-reported?symbol={stock}&metric=all&token={FINNHUB_API_KEY}'

        pricing_data = requests.get(pricing_api_url).json()
        if pricing_data.get('error'):
            raise Exception(f"Error fetching pricing data for {stock}: {pricing_data['error']}")

        basic_financials = requests.get(basic_financials_api_url).json()
        if basic_financials.get('error'):
            raise Exception(f"Error fetching basic financials data for {stock}: {basic_financials['error']}")

        financials_reported = requests.get(financials_reported_api_url).json()
        if financials_reported.get('error'):
            raise Exception(f"Error fetching financials reported data for {stock}: {financials_reported['error']}")

        price = pricing_data.get('c') or 'N/A'
        pe_annual = basic_financials.get('metric', {}).get('peAnnual') or 'N/A'
        pb = basic_financials.get('metric', {}).get('pb') or 'N/A'
        ps_annual = basic_financials.get('metric', {}).get('psAnnual') or 'N/A'
        ev_ebitda_ttm = basic_financials.get('metric', {}).get('evEbitdaTTM') or 'N/A'

        ev = basic_financials.get('metric', {}).get('enterpriseValue')* 1000000 or 'N/A'
        income_statement = financials_reported.get('data', [])[0].get('report', {}).get('ic', []) or 'N/A'

        gross_profit_value = (
            next((item.get('value') for item in income_statement
                if item.get('concept') == 'us-gaap_GrossProfit'), 'N/A')
            if isinstance(income_statement, list) else 'N/A'
        )

        ev_gross_profit = ev / gross_profit_value if ev != 'N/A' and gross_profit_value != 'N/A' and gross_profit_value != 0 else 'N/A'

    except Exception as e:
        print(f"An error occurred for {stock}: {e}")
        price = 'N/A'
        one_year_price_return = 'N/A'
    
    rows.append([stock, price, 'N/A', pe_annual, 'N/A', pb, 'N/A', ps_annual, 'N/A', ev_ebitda_ttm, 'N/A', ev_gross_profit, 'N/A', 'N/A'])
    
    time.sleep(2)

final_dataframe = pd.DataFrame(rows, columns = rv_columns)

final_dataframe

An error occurred for ALXN: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for ANTM: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for BLL: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for BRK.B: list index out of range
An error occurred for CERN: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for COG: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for CTL: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for CXO: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for DISCA: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for DISCK: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for ETFC: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for FB: unsupported operand type(s) for *: 'NoneType' and 'int'
An error occurred for FBHS:

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Gross Profit Percentile,RV Score
0,A,137.95,N/A,30.0143,N/A,6.1395,N/A,5.6288,N/A,23.3653,N/A,N/A,N/A,N/A
1,AAL,15.48,N/A,12.0782,N/A,63.8634,N/A,0.1885,N/A,8.5484,N/A,N/A,N/A,N/A
2,AAP,38.89,N/A,N/A,N/A,1.0634,N/A,0.2567,N/A,5.7138,N/A,0.754364,N/A,N/A
3,AAPL,271.01,N/A,35.7516,N/A,54.3114,N/A,9.6226,N/A,28.1612,N/A,20.836276,N/A,N/A
4,ABBV,229.31,N/A,94.7356,N/A,285.4077,N/A,7.1942,N/A,92.3306,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,150.49,N/A,28.1184,N/A,267.846,N/A,5.535,N/A,19.4325,N/A,N/A,N/A,N/A
501,ZBH,90.02,N/A,19.7414,N/A,1.3985,N/A,2.3234,N/A,12.6443,N/A,N/A,N/A,N/A
502,ZBRA,248.3,N/A,23.8237,N/A,3.3571,N/A,2.5254,N/A,15.7459,N/A,5.67714,N/A,N/A
503,ZION,59.25,N/A,11.1578,N/A,1.2743,N/A,1.6145,N/A,11.4384,N/A,N/A,N/A,N/A


In [99]:
final_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Gross Profit Percentile,RV Score
0,A,137.95,N/A,30.0143,N/A,6.1395,N/A,5.6288,N/A,23.3653,N/A,N/A,N/A,N/A
1,AAL,15.48,N/A,12.0782,N/A,63.8634,N/A,0.1885,N/A,8.5484,N/A,N/A,N/A,N/A
2,AAP,38.89,N/A,N/A,N/A,1.0634,N/A,0.2567,N/A,5.7138,N/A,0.754364,N/A,N/A
3,AAPL,271.01,N/A,35.7516,N/A,54.3114,N/A,9.6226,N/A,28.1612,N/A,20.836276,N/A,N/A
4,ABBV,229.31,N/A,94.7356,N/A,285.4077,N/A,7.1942,N/A,92.3306,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,150.49,N/A,28.1184,N/A,267.846,N/A,5.535,N/A,19.4325,N/A,N/A,N/A,N/A
501,ZBH,90.02,N/A,19.7414,N/A,1.3985,N/A,2.3234,N/A,12.6443,N/A,N/A,N/A,N/A
502,ZBRA,248.3,N/A,23.8237,N/A,3.3571,N/A,2.5254,N/A,15.7459,N/A,5.67714,N/A,N/A
503,ZION,59.25,N/A,11.1578,N/A,1.2743,N/A,1.6145,N/A,11.4384,N/A,N/A,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [114]:
final_dataframe_updated = final_dataframe[(final_dataframe['Price'] != 'N/A') & (final_dataframe['Price-to-Earnings Ratio'] != 'N/A') & (final_dataframe['Price-to-Book Ratio'] != 'N/A') & (final_dataframe['Price-to-Sales Ratio'] != 'N/A') & (final_dataframe['EV/EBITDA'] != 'N/A') & (final_dataframe['EV/Gross Profit'] != 'N/A')]

# final_dataframe_updated = final_dataframe_updated.sort_values('One-Year Price Return', ascending=False)
# final_dataframe_updated = final_dataframe_updated[:50]
final_dataframe_updated.reset_index(inplace=True, drop=True)
# final_dataframe_updated

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [ ]:
for column in final_dataframe_updated.columns:
    final_dataframe_updated[column + ' Percentile'] = final_dataframe_updated[column].rank(pct=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [ ]:
# Not a fan of this approach so won't be using it further - just dropping the rows with N/A values instead of taking the average of the column as suggested in the course (because the data would be inaccurate with simply taking the average)

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [124]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/Gross Profit': 'EV/Gross Profit Percentile'
}

# for metric in metrics.keys():
#     for row in final_dataframe_updated.index:
#         final_dataframe_updated.loc[row, metrics[metric]] = stats.percentileofscore(final_dataframe_updated[metric], final_dataframe_updated.loc[row, metric])

# Computing percentiles vectorially using pandas’ built-in functions to operate on entire columns at once, instead of looping through each row and assigning values individually.

for metric, out_col in metrics.items():
    final_dataframe_updated[out_col] = final_dataframe_updated[metric].rank(pct=True)

final_dataframe_updated

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Gross Profit Percentile,RV Score
0,AAPL,271.01,N/A,35.7516,0.659091,54.3114,0.924242,9.6226,0.878788,28.1612,0.803030,20.836276,0.909091,N/A
1,ADBE,333.3,N/A,19.5679,0.250000,12.0037,0.757576,5.8698,0.696970,15.3988,0.310606,7.327434,0.393939,N/A
2,ADI,273.74,N/A,59.1168,0.886364,3.9638,0.378788,12.1635,0.924242,38.4486,0.878788,20.856147,0.916667,N/A
3,ADM,59.05,N/A,15.7654,0.159091,1.2616,0.106061,0.3318,0.030303,14.8425,0.272727,6.057225,0.287879,N/A
4,ADSK,286.73,N/A,54.6644,0.863636,21.0117,0.840909,9.9147,0.893939,39.0566,0.893939,11.035433,0.628788,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,WDC,187.7,N/A,33.9727,0.621212,10.4912,0.734848,6.741,0.780303,20.0931,0.553030,18.095748,0.833333,N/A
128,WST,276.4,N/A,40.3596,0.734848,6.5165,0.590909,6.8731,0.803030,25.6961,0.765152,19.494001,0.871212,N/A
129,WY,23.8,N/A,43.1242,0.787879,1.8094,0.151515,2.3971,0.378788,21.564,0.628788,16.866867,0.803030,N/A
130,XYL,137.11,N/A,37.5079,0.681818,2.9787,0.250000,3.8989,0.568182,22.9523,0.689394,10.632951,0.598485,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [128]:
for row in final_dataframe_updated.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(final_dataframe_updated.loc[row, metrics[metric]])
    
    final_dataframe_updated.loc[row, 'RV Score'] = mean(value_percentiles)

final_dataframe_updated

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Gross Profit Percentile,RV Score
0,AAPL,271.01,N/A,35.7516,0.659091,54.3114,0.924242,9.6226,0.878788,28.1612,0.803030,20.836276,0.909091,0.834848
1,ADBE,333.3,N/A,19.5679,0.250000,12.0037,0.757576,5.8698,0.696970,15.3988,0.310606,7.327434,0.393939,0.481818
2,ADI,273.74,N/A,59.1168,0.886364,3.9638,0.378788,12.1635,0.924242,38.4486,0.878788,20.856147,0.916667,0.79697
3,ADM,59.05,N/A,15.7654,0.159091,1.2616,0.106061,0.3318,0.030303,14.8425,0.272727,6.057225,0.287879,0.171212
4,ADSK,286.73,N/A,54.6644,0.863636,21.0117,0.840909,9.9147,0.893939,39.0566,0.893939,11.035433,0.628788,0.824242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,WDC,187.7,N/A,33.9727,0.621212,10.4912,0.734848,6.741,0.780303,20.0931,0.553030,18.095748,0.833333,0.704545
128,WST,276.4,N/A,40.3596,0.734848,6.5165,0.590909,6.8731,0.803030,25.6961,0.765152,19.494001,0.871212,0.75303
129,WY,23.8,N/A,43.1242,0.787879,1.8094,0.151515,2.3971,0.378788,21.564,0.628788,16.866867,0.803030,0.55
130,XYL,137.11,N/A,37.5079,0.681818,2.9787,0.250000,3.8989,0.568182,22.9523,0.689394,10.632951,0.598485,0.557576


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [ ]:
final_dataframe_updated.sort_values('RV Score', ascending=True, inplace=True)
final_dataframe_updated = final_dataframe_updated[:50].reset_index(drop=True)
final_dataframe_updated

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Gross Profit Percentile,RV Score
0,HPQ,22.12,N/A,8.0292,0.037879,0.7313,0.037879,0.3672,0.053030,7.618,0.045455,2.305725,0.037879,0.042424
1,PVH,67.82,N/A,5.1902,0.007576,0.6367,0.022727,0.359,0.045455,12.3329,0.151515,1.012531,0.007576,0.04697
2,MHK,109.51,N/A,13.0772,0.121212,0.8122,0.045455,0.6247,0.098485,6.8881,0.022727,3.0408,0.068182,0.071212
3,LKQ,30.03,N/A,11.1401,0.106061,1.1728,0.090909,0.5355,0.090909,7.4707,0.037879,2.058396,0.030303,0.071212
4,NOV,16.41,N/A,9.4261,0.060606,0.9273,0.068182,0.6748,0.106061,7.3728,0.030303,3.370943,0.090909,0.071212
5,TAP,47.4,N/A,8.3684,0.045455,0.9097,0.053030,0.8078,0.159091,7.9975,0.053030,3.250214,0.083333,0.078788
6,RHI,27.34,N/A,10.9955,0.090909,2.151,0.166667,0.4773,0.083333,15.3257,0.303030,1.068492,0.015152,0.131818
7,BBY,69.18,N/A,15.6372,0.151515,5.4639,0.500000,0.3491,0.037879,11.2673,0.083333,1.57034,0.022727,0.159091
8,ADM,59.05,N/A,15.7654,0.159091,1.2616,0.106061,0.3318,0.030303,14.8425,0.272727,6.057225,0.287879,0.171212
9,KHC,24.39,N/A,10.5209,0.083333,0.6965,0.030303,1.117,0.196970,18.1894,0.446970,5.346494,0.189394,0.189394


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [134]:
portfolio_input()

In [139]:
position_size = float(portfolio_size) / len(final_dataframe_updated.index)
for row in final_dataframe_updated.index:
    final_dataframe_updated.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe_updated.loc[row, 'Price'])

final_dataframe_updated

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Gross Profit Percentile,RV Score
0,HPQ,22.12,2260,8.0292,0.037879,0.7313,0.037879,0.3672,0.053030,7.618,0.045455,2.305725,0.037879,0.042424
1,PVH,67.82,737,5.1902,0.007576,0.6367,0.022727,0.359,0.045455,12.3329,0.151515,1.012531,0.007576,0.04697
2,MHK,109.51,456,13.0772,0.121212,0.8122,0.045455,0.6247,0.098485,6.8881,0.022727,3.0408,0.068182,0.071212
3,LKQ,30.03,1665,11.1401,0.106061,1.1728,0.090909,0.5355,0.090909,7.4707,0.037879,2.058396,0.030303,0.071212
4,NOV,16.41,3046,9.4261,0.060606,0.9273,0.068182,0.6748,0.106061,7.3728,0.030303,3.370943,0.090909,0.071212
5,TAP,47.4,1054,8.3684,0.045455,0.9097,0.053030,0.8078,0.159091,7.9975,0.053030,3.250214,0.083333,0.078788
6,RHI,27.34,1828,10.9955,0.090909,2.151,0.166667,0.4773,0.083333,15.3257,0.303030,1.068492,0.015152,0.131818
7,BBY,69.18,722,15.6372,0.151515,5.4639,0.500000,0.3491,0.037879,11.2673,0.083333,1.57034,0.022727,0.159091
8,ADM,59.05,846,15.7654,0.159091,1.2616,0.106061,0.3318,0.030303,14.8425,0.272727,6.057225,0.287879,0.171212
9,KHC,24.39,2050,10.5209,0.083333,0.6965,0.030303,1.117,0.196970,18.1894,0.446970,5.346494,0.189394,0.189394


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [169]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
final_dataframe_updated.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [170]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [171]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/Gross Profit', float_template],
    'M': ['EV/Gross Profit Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [172]:
writer.close()